## 바른 형태소 분석기 활용 5(토픽 모델링)
(이미 설치를 완료하신 분들은 아래 '샘플 텍스트 파일 다운로드' 부분부터 실행하면 됩니다.)

바른 설치
바른을 설치합니다.

바른의 리눅스 설치본을 다운로드 받습니다.

In [2]:
!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

bareun-linux.deb			     nohup.out	  summary_sentences.txt
libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz  sample_data


In [3]:
!uname -a

Linux 2fc4d1c2e495 5.15.109+ #1 SMP Fri Jun 9 10:57:30 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


**바른**을 설치합니다.

In [4]:
!dpkg -i bareun-linux.deb

(Reading database ... 120694 files and directories currently installed.)
Preparing to unpack bareun-linux.deb ...
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Removed /etc/systemd/system/multi-user.target.wants/bareun.service.
Unpacking bareun (2.1.0) over (2.1.0) ...
Setting up bareun (2.1.0) ...
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Created symlink /etc/systemd/system/multi-user.target.wants/bareun.service → /etc/systemd/system/bareun.service.


아래는 바른 모델이 GPU를 사용하기 위해서 다운받는 tensorflow 라이브러리입니다.

상단의 메뉴탭의 "런타임 > 런타임 유형 변경 > 하드웨어 가속기 > GPU 선택 > 저장"을 설정하시면 모델의 추론 속도가 더 빨라집니다. 단, 무료버전의 경우 coloab의 일일 사용 제한을 초과하지 않아야 합니다.

기본적으로 CPU를 사용해도 작동합니다.

In [5]:
!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  392M  100  392M    0     0   211M      0  0:00:01  0:00:01 --:--:--  210M


In [6]:
!tar -C /opt/bareun/ -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

In [7]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## 환경 설정하기
아래 코드는 BAREUN_ROOT와 LD_LIBRARY_PATH를 설정하고, /opt/bareun/bin/bareun을 백그라운드로 실행하는 코드입니다.

In [8]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&


env: BAREUN_ROOT="/opt/bareun"
env: LD_LIBRARY_PATH="/opt/bareun/lib"
nohup: appending output to 'nohup.out'


In [9]:
!ps -ef | grep bareun

root        4274       1 47 01:12 ?        00:09:26 /opt/bareun/bin/bareun
root        9482       1  0 01:31 ?        00:00:00 /opt/bareun/bin/bareun
root        9483    8383  0 01:31 ?        00:00:00 /bin/bash -c ps -ef | grep bareun
root        9485    9483  0 01:31 ?        00:00:00 grep bareun


API키 발급 바른 홈페이지(https://bareun.ai/) 에 회원가입해서 API키를 받습니다.

API키 복사
발급 받은 API키를 -reg 인자 다음에 입력하여 등록합니다.

In [ ]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg "YOUR-API-KEY"

## 바른 사용하기
**PIP 패키지 설치**


In [11]:
!pip install -U bareunpy

In [ ]:
import sys
from bareunpy import Tagger
import google.protobuf.text_format as tf
API_KEY = "YOUR-API-KEY"
tagger = Tagger(API_KEY, "localhost", 5656)

## 샘플 텍스트 파일 다운로드

* 약 8,600개의 문장으로 구성된 샘플 텍스트 파일입니다.
* Aihub 기술과학요약 데이터의 일부를 추출했습니다.

In [13]:
!pip install -U gdown

In [14]:
!gdown  https://drive.google.com/file/d/14Qs9KI68z0RdJ9q8G__2fgsv0jNI43m_/view?usp=sharing -O summary_sentences.txt --fuzzy

Downloading...
From: https://drive.google.com/uc?id=14Qs9KI68z0RdJ9q8G__2fgsv0jNI43m_
To: /content/summary_sentences.txt
100% 4.21M/4.21M [00:00<00:00, 167MB/s]


In [15]:
# 필요한 패키지 설치
!pip install tqdm gensim pandas==1.5.3 pyLDAvis

## 토픽 모델링 실행하기

In [16]:
import re
from tqdm import tqdm

from gensim import corpora
import gensim
import pyLDAvis.gensim_models

In [17]:
def preprocess_text(lines, tagger):
    """
    불용어를 제거하기 위해 토큰화한다.
    :param lines: 문장 리스트
    :param tagger: 형태소 분석 객체
    :return: 토큰화된 문장 리스트
    """
    tokenized_lines = []
    for line in tqdm(lines):
        # 한글만 추출하는 정규표현식
        line = re.sub('[^가-힣]', '', line)

        tagged = tagger.tags([line], auto_spacing=False, auto_jointing=False)
        tokens = []
        # 명사만 추출
        for p in tagged.pos():
          if (p[1] == 'NNG' or p[1] == 'NNP') and len(p[0]) > 1: # 한 글자 단어 제외
                tokens.append(p[0])
        tokenized_lines.append(tokens)

    return tokenized_lines

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 1. corpus 준비하기
* 기술과학 요약 데이터에서 추출한 text 파일을 불러옵니다.

In [18]:
with open('summary_sentences.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
print(lines[:5])

['스마트 팩토리 구축을 위해서 다양한 센서들을 기존의 공정 설비 및 제조 장비 내에 설치하는 과정이 필요하며, 센서들로부터 수집한 데이터등은 서버로 전송된다. 이때 스마트 팩토리를 구축하려면, 기존의 공정 설비 또는 제조 장비들을 커스터마이징하거나 새로운 설비 및 장비의 개발의 선행이 이루어져야한다. 논문에서는 중소기업에서 원격 제어 시스템을 통해 단계적으로 기능을 추가하는 스마트 팩토리 환경 구축을 제안한다. 특히 따라서 원격 제어 시스템 중 CNC절단 장비를 대상으로 제안하며, 기존 CNC 절단 장비와의 호환성을 위해 절단 장비 내 PLC의 연동에 초점을 맞추고자 한다.\n', '서버와 데이터베이스는 Node.js와 MySQL을 활용하고 자바를 사용한 Data collection module과 Control module을 구현한 CNC 절단 장비를 이용하여 원격 제어 시스템의 구조 및 설계를 검증하는데 영상데이터 재생을 지원하는 방법으로 웹페이지에서 CNC 절단 장비의 현재 상태 모니터링 페이지를 나타내고, 동일한 기능을 제공하도록 안드로이드 어플리케이션을 같이 개발하였다.\n', '국내외 여러 회사들이 유무선네트워크 및 센서 등의 기술과 기계학습으로 대표되는 지능화 기술이 발전함에 따라 설비운영의 효율화, 비용절감, 새로운 제품 및 서비스 제공의 이점에 의해 빅데이터 및 기계 학습 기반의 지능화를 통한 자동적인 공정 제어를 목표로 하는 스마트 팩토리로의 전환을 진행하고 있다.\n', '제조 장비는 Data collection module과 Control module 로 구성되며, 다음 구성요소인 서버는 Data collection module과의 통신을 통해 센싱 데이터를 수신 받아 데이터베이스에 저장하고 모니터링 프로그램에 CNC 절단 장비의 상태 정보를 제공하며, 마지막 구성요소로서 웹과 안드로이드 어플리케이션을 통해 사용자에게 제조 장비의 현재 상태 모니터링 기능과 장비의 제어 명령 인터페이스를 제공한다.\n', 'Controlmodule은 사설 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 2 형태소 분석 및 명사 추출

* preprocess_text 함수를 실행하여 바른 형태소 분석을 수행하고 명사만을 추출합니다.

In [22]:
tokenized_lines = preprocess_text(lines, tagger)
print(tokenized_lines[:5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 8647/8647 [06:41<00:00, 21.52it/s]

[['스마트', '팩토리', '구축', '다양', '센서', '기존', '공정', '설비', '제조', '장비', '설치', '과정', '필요', '센서', '수집', '데이터', '서버', '전송', '다이때', '스마트', '팩토리', '구축', '하려면', '기존', '공정', '설비', '제조', '장비', '커스터', '마이징', '설비', '장비', '개발', '선행', '논문', '중소기업', '원격', '제어', '시스템', '단계', '기능', '추가', '스마트', '팩토리', '환경', '구축', '제안', '원격', '제어', '시스템', '중절단', '장비', '대상', '제안', '기존', '절단', '장비', '호환', '절단', '장비', '연동', '초점'], ['서버', '데이터베이스', '와을', '활용', '사용', '구현', '절단', '장비', '이용', '원격', '제어', '시스템', '구조', '설계', '검증', '영상', '데이터', '재생', '지원', '방법', '페이지', '절단', '장비', '현재', '상태', '모니터링페이지', '동일', '기능', '제공', '안드로이드어플리케이션', '개발'], ['국내외', '회사', '유무선', '네트워크', '센서', '기술', '기계', '학습', '대표', '지능', '기술', '발전', '설비', '운영', '효율', '비용', '절감', '제품', '서비스', '제공', '이점', '빅데', '기계', '학습', '기반', '지능', '자동', '공정', '제어', '목표', '스마트', '팩토리', '전환', '진행'], ['제조', '장비', '구성', '다음', '구성', '요소', '서버', '통신', '센싱데이터', '수신', '데이터베이스', '저장', '모니터링', '프로그램', '절단', '장비', '상태', '정보', '제공', '마지막', '구성요소', '안드로이드어플리케이션', '사용자', '제조', '장비', '현재', '상태', 

### 3. 정수 인코딩 및 단어 집합 생성
* gensim의 Dictionary 객체를 생성합니다. 이 객체는 텍스트 데이터의 각 단어에 고유한 정수 ID를 할당합니다.
* 그리고 각 문서를 'bag-of-words'형식으로 변환합니다.
* 'bag-of-words'는 텍스트 데이터를 숫자로 표현하는 일반적인 방법으로 각 단어의 빈도를 나타냅니다.

In [23]:
dictionary = corpora.Dictionary(tokenized_lines)
corpus = [dictionary.doc2bow(text) for text in tokenized_lines]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 4. LDA 모델 훈련시키기

* corpus와 dictionary는 위에서 생성한 것을 사용합니다.
* num_topics는 모델이 찾을 주제의 수를 지정하며, passes는 훈련 과정에서 데이터를 반복하는 횟수입니다. ldamodel.print_topics(num_words=4)를 통해 각 주제의 상위 4개 단어를 출력합니다.

In [24]:
NUM_TOPICS = 3 # 주제의 수
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = lda_model.print_topics(num_words=4)
for topic in topics:
    print(f'{topic[0]}번째 토픽: {topic[1]}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0번째 토픽: 0.013*"사용" + 0.012*"데이터" + 0.011*"시스템" + 0.011*"제안"
1번째 토픽: 0.076*"함수" + 0.031*"집합" + 0.018*"정의" + 0.015*"정리"
2번째 토픽: 0.038*"행렬" + 0.030*"확률" + 0.022*"벡터" + 0.016*"분포"


### 5. LDA 시각화하기

* pyLDAvis라는 패키지를 사용하여 LDA 모델의 결과를 시각화합니다.
* 이 패키지는 주제 간의 거리와 각 주제에서 가장 중요한 단어를 시각적으로 보여줍니다.

In [25]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)



### 6. 분석하기

* 위 시각화 자료는 키워드 간 연관성을 기반으로 그룹화되었습니다.

#### 토픽의 수와 퀄리티 확인

* 각 토픽에 대한 키워드를 보고 합리적인 주제를 형성하고 있는지, 유용한 인사이트를 제공하는지 판단할 수 있습니다.

#### 토픽 간 거리 확인

* 토픽들 사이의 거리를 확인할 수 있습니다. 이 거리는 토픽들이 얼마나 서로 다른지를 나타내며, 서로 가까운 토픽은 유사한 주제를 다루고 있다고 볼 수 있습니다.

#### 토픽 분포 확인

* 각 문서에서 어떤 토픽이 가장 크게 기여하고 있는지 확인함으로써, 그 문서의 주제나 내용을 이해할 수 있습니다.

#### 토픽 내의 주요 단어 분석

* 각 주제에서 그룹화된 키워드들을 확인합니다.
* 각 주제별로 "어떤 키워드들로 묶였는지?", "몇 개의 그룹으로 묶어야 하는지?", "숨겨진 주제를 찾을 수도 있는지?" 등의 질문을 던지며 파라미터를 수정하고, 목적에 맞게 데이터를 분석합니다.
* 해당 코퍼스는 기술과학요약 데이터로, "자연"과 "인공물" 두 가지 field에서 추출한 데이터입니다.
* 그렇다면 큰 토픽은 두 가지라고 볼 수 있습니다. 토픽을 모른다고 가정하고, 세 가지로 분류했을 때, 현재 그림에서 2번과 3번 토픽이 거리가 가깝고 나오는 키워드들이 서로 유사하다면(집합, 행렬, 벡터, 함수, 방정식, ...) 2번과 3번을 하나의 토픽으로도 묶일 수 없는지 보고, 파라미터를 수정해볼 수 있을 것 같습니다.